In [1]:
import numpy as np
import pandas as pd
from pandas import Series
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn')
sns.set(font_scale=2.5)

In [3]:
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

import warnings
warnings.filterwarnings('ignore')

In [4]:
%matplotlib inline

df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')

df_train['FamilySize'] = df_train['SibSp'] + df_train['Parch'] + 1 #  + 1 : 자신
df_test['FamilySize'] = df_test['SibSp'] + df_test['Parch'] + 1

# null값 평균값 대치
df_test.loc[df_test.Fare.isnull(), 'Fare'] = df_test['Fare'].mean()

df_train['Fare'] = df_train['Fare'].map(lambda i : np.log(i) if i > 0 else 0)
df_test['Fare'] = df_test['Fare'].map(lambda i : np.log(i) if i > 0 else 0)

### 3.1 Fill Null
#### 3.1.1 Fill Null in Age using title
* Age 에는 null data가 177개나 있습니다. 이를 채울 수 있는 여러 아이디어가 있을 것인데, 여기서 우리는 title + statistics 를 사용해 보겠습니다.
* 영어에는 Miss, Mrr, Mrs 같은 title 이 존재합니다. 각 탑승객의 이름에는 꼭 이런 title 이 들어가게 되는데 이를 사용해보겠습니다.
* pandas series 에는 data를 string 으로 바꿔주는 strmethod, 거기에 정규포현식을 적용하ㅏ게 해주는 extractmethod 가 있습니다. 이를 사용하여 title 을 쉽게 추출할 수 있습니다. title 을 Initial column에 저장하겠습니다.

In [6]:
# lets extract the salutations
df_train['Initial'] = df_train.Name.str.extract('([A-Za-z]+)\.')
df_train['Initial'] = df_test.Name.str.extract('([A-Za-z]+)\.')

* pandas 의 crosstab 을 이용하여 우리가 추출한 Initial 과 Sex 간의 count 를 살펴봅시다.

In [7]:
pd.crosstab(df_train['Initial'], df_train['Sex']).T.style.background_gradient(cmap='summer_r')

Initial,Col,Dona,Dr,Master,Miss,Mr,Mrs,Ms,Rev
Sex,,,,,,,,,
female,0,0,1,6,26,95,31,1,0
male,2,1,0,15,52,145,41,0,2


* 위 table 을 참고하여, 남자, 여자가 쓰는 initial 을 구분해 보겠습니다. replace 메소드를 사용하면, 특정 데이터 값을 원하는 값으로 치환해줍니다. 